In [ ]:
import kfp  # the Pipelines SDK. 
from kfp import compiler
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.components as comp

In [ ]:
tb_op = comp.load_component_from_url(
  'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/tensorflow/tensorboard/prepare_tensorboard/component.yaml' 
  )
train_op = comp.load_component_from_url(
  'https://gitlab.cern.ch/ai-ml/examples/-/raw/master/tfjob_tensorboard/training_component/training_comp.yaml'
  )
serve_op = comp.load_component_from_url(
  'https://gitlab.cern.ch/ai-ml/examples/-/raw/master/tfjob_tensorboard/serving_component/serving_comp.yaml'
  )

In [ ]:
@dsl.pipeline(
  name='bikes_weather_metrics',
  description='Model bike rental duration given weather'
)
def bikes_weather_metrics( 
    train_epochs: int = 2,
    working_dir: str = 'gs://YOUR/GCS/PATH',  # for the full training jobs
    data_dir: str = 'gs://aju-dev-demos-codelabs/bikes_weather/',
    steps_per_epoch: int = 2 ,  # if -1, don't override normal calcs based on dataset size
    hptune_params: str = '[{"num_hidden_layers": %s, "learning_rate": %s, "hidden_size": %s}]' % (3, 1e-2, 64),
    thresholds: str = '{"root_mean_squared_error": 10000}'
  ):

    tb_viz = tb_op(
        log_dir_uri='%s/%s' % (working_dir, dsl.RUN_ID_PLACEHOLDER)
    )

    train = train_op(
        data_dir=data_dir,
        workdir='%s/%s' % (tb_viz.outputs['log_dir_uri'], 0),
        tb_dir=tb_viz.outputs['log_dir_uri'],
        epochs=train_epochs, 
        steps_per_epoch=steps_per_epoch,
        hp_idx=0, 
        hptune_results=hptune_params,
        tfjob_image="gitlab-registry.cern.ch/ai-ml/kubeflow_images/training_tfjob:20"
    )

    serve = serve_op(
      model_path=train.outputs['train_output_path'],
      model_name='bikesw'
      )

In [ ]:
# CHANGE THIS with the info for your KFP cluster installation
client = kfp.Client()

exp = client.create_experiment(name='bw_expers')  # this is a 'get or create' call

WORKING_DIR = 'gs://dejan_testing_bucket'
TRAIN_EPOCHS = 2

compiler.Compiler().compile(bikes_weather_metrics, 'bikes_weather_metrics.tar.gz')

#client.upload_pipeline('bikes_weather_metrics.tar.gz', 'pip_bikes_dg_15', 'namespace: dejan-golubovic')

run = client.run_pipeline(exp.id, 'bw_metrics_test', 'bikes_weather_metrics.tar.gz',
                          params={'working_dir': WORKING_DIR, 'train_epochs': TRAIN_EPOCHS
                                 # 'thresholds': THRESHOLDS
                                 })